In [1]:
import pandas as pd

tour_points = pd.read_csv('tbbp23_bs_16_202506251117.csv')
accommodations = pd.read_csv('filtered_data.csv')

In [2]:
tour_points.head()

,LOC_NM,LATITUDE,LONGITUDE
0,40계단,35.104343,129.035340
1,40계단테마거리,35.103972,129.034629
2,BIFF광장,35.098620,129.028759
3,광복로,35.099470,129.031166
4,백산기념관,35.101955,129.034560


In [7]:
import numpy as np

In [8]:
숙박 = pd.read_csv('filtered_data.csv')
명소 = pd.read_csv('tbbp23_bs_16_202506251117.csv')

RADIUS_KM = 1.0

nearby_counts = []

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    c = 2*np.arcsin(np.sqrt(a))
    return R*c

for i, 숙_row in 숙박.iterrows():
    lat1 = 숙_row['위도']
    lon1 = 숙_row['경도']
    distances = haversine(lat1, lon1, 명소['LATITUDE'].values, 명소['LONGITUDE'].values)
    count = np.sum(distances <= RADIUS_KM)
    nearby_counts.append(count)

숙박['주변명소 수']=nearby_counts
    

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
숙박['숙박점수_norm'] = scaler.fit_transform(숙박[['숙박점수']])
숙박['명소수_norm'] = scaler.fit_transform(숙박[['주변명소 수']])

숙박['숙박점수_역'] = 1-숙박['숙박점수_norm']
숙박['종합점수'] = 0.5*숙박['숙박점수_역'] + 0.5*숙박['명소수_norm']

In [15]:
숙박['주변명소 수'].value_counts()

주변명소 수
0     1020
4      230
7      198
5       84
47      83
1       63
9       60
6       53
3       47
10      46
11      36
44      21
8       15
38      11
41      10
13      10
35       9
29       6
23       6
17       6
2        6
27       6
32       5
22       4
16       4
20       4
26       3
18       3
14       2
28       2
15       1
37       1
40       1
12       1
25       1
43       1
19       1
Name: count, dtype: int64

숙박 행들 중 주변명소 수 가 0인 행이 1020개에 해당했음. 이대로 분석을 진행한다면 전체 분석의 신뢰도와 로직에 큰 영향을 미칠 것으로 예상되므로, 별도의 처리가 필요할 것으로 판단.
우선 이 데이터가 의미하는 것은 1km 반경 내에 관광명소가 전혀 없는 숙박시설이 1020곳이었다, 그럼에도 이 숙박업소들 중 일부는 여전히 우수한 친환경 점수 후보일 수 있다. 그러나 주변 명소가 0개이면 관광 편의성 측면에서 추천도는 낮아야 한다. 이 세 가지 측면을 모두 고려해 다시 점수를 매겨야 함.

In [17]:
# option 1. 명소 수가 0이면 종합 점수에 불이익을 주는 방식. 가장 직관적, 사용자 기대와도 부합. 별도 처리가 없어도 자동으로 주변 명소 없는 숙소는 종합점수 낮아짐.
숙박['명소수_norm'] = scaler.fit_transform(숙박[['주변명소 수']])

숙박['종합점수'] = 0.5*(1-숙박['숙박점수_norm'])+0.5*숙박['명소수_norm']

In [ ]:
# option 2. 주변 명소 수가 0이더라도 친환경 점수가 매우 좋으면 예외적으로 추천. 친환경 점수 우수+명소 없음->지속가능 관광지 후보로 따로 분류
숙박['추천분류'] = 숙박.apply(lambda row:
                      '최우수' if row['종합점수'] >= 0.8 else (
                          '친환경_단독' if row['주변명소 수'] == 0 and row['숙박점수'] <= 숙박['숙박점수'].quantile(0.1)
                          else '일반'), axis=1)

In [18]:
숙박.head()

,업체명,경도,위도,숙박유형,숙박점수,주변명소 수,숙박점수_norm,명소수_norm,숙박점수_역,종합점수
0,베스티호텔,128.853975,35.128196,호텔/리조트,28.03,0,1.000000,0.0,0.000000,0.000000
1,대저캠핑장,128.991081,35.215947,캠핑/카라반/글램핑,7.98,0,0.000000,0.0,1.000000,0.500000
2,공항하숙,128.955791,35.180705,펜션/민박/유스호스텔,8.51,0,0.026434,0.0,0.973566,0.486783
3,에어포트호텔,128.954507,35.165692,호텔/리조트,28.03,0,1.000000,0.0,0.000000,0.000000
4,초롱블루펜션,128.834494,35.015366,펜션/민박/유스호스텔,8.51,0,0.026434,0.0,0.973566,0.486783


In [20]:
숙박.to_csv('숙박_종합친환경점수.csv', index=False, encoding='utf-8-sig')

In [21]:
# option 3. 반경 확장
숙박2 = pd.read_csv('filtered_data.csv')
명소 = pd.read_csv('tbbp23_bs_16_202506251117.csv')

숙박3 = pd.read_csv('filtered_data.csv')


RADIUS_KM_2 = 2.0
RADIUS_KM_3 = 3.0

nearby_counts = []

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    c = 2*np.arcsin(np.sqrt(a))
    return R*c

for i, 숙_row in 숙박2.iterrows():
    lat1 = 숙_row['위도']
    lon1 = 숙_row['경도']
    distances = haversine(lat1, lon1, 명소['LATITUDE'].values, 명소['LONGITUDE'].values)
    count = np.sum(distances <= RADIUS_KM_2)
    nearby_counts.append(count)
숙박2['주변명소 수']=nearby_counts

nearby_counts = []
for i, 숙_row in 숙박3.iterrows():
    lat1 = 숙_row['위도']
    lon1 = 숙_row['경도']
    distances = haversine(lat1, lon1, 명소['LATITUDE'].values, 명소['LONGITUDE'].values)
    count = np.sum(distances <= RADIUS_KM_3)
    nearby_counts.append(count)

숙박3['주변명소 수']=nearby_counts

In [22]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
숙박2['숙박점수_norm'] = scaler.fit_transform(숙박2[['숙박점수']])
숙박2['명소수_norm'] = scaler.fit_transform(숙박2[['주변명소 수']])

숙박2['숙박점수_역'] = 1-숙박2['숙박점수_norm']
숙박2['종합점수'] = 0.5*숙박2['숙박점수_역'] + 0.5*숙박2['명소수_norm']

In [23]:
숙박2['주변명소 수'].value_counts()

주변명소 수
0     698
15    133
8     121
59    117
16    104
4     101
18     87
5      76
13     62
51     52
34     48
27     45
9      32
56     28
12     28
14     26
19     22
30     21
10     20
47     19
6      17
11     16
46     16
3      15
50     15
24     15
1      14
31     13
43     12
28     11
20     10
17      9
60      6
23      6
54      6
37      4
65      4
21      4
53      4
62      3
40      3
61      2
35      2
26      2
52      2
64      2
7       2
63      1
48      1
2       1
29      1
33      1
Name: count, dtype: int64

In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
숙박3['숙박점수_norm'] = scaler.fit_transform(숙박3[['숙박점수']])
숙박3['명소수_norm'] = scaler.fit_transform(숙박3[['주변명소 수']])

숙박3['숙박점수_역'] = 1-숙박3['숙박점수_norm']
숙박3['종합점수'] = 0.5*숙박3['숙박점수_역'] + 0.5*숙박3['명소수_norm']

In [25]:
숙박3['주변명소 수'].value_counts()

주변명소 수
0     507
22    169
8     127
4      83
47     60
     ... 
11      1
7       1
27      1
64      1
14      1
Name: count, Length: 65, dtype: int64